# Movie Recommendation

In [1]:
import pandas as pd
import math
import mlflow
import os

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, mean, udf, lit, current_timestamp, unix_timestamp, array_contains
from pyspark.sql.types import IntegerType, FloatType

## Converting Data to RDD

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive recommender") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

22/11/20 18:55:23 WARN Utils: Your hostname, sahil-dell resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface wlp2s0)
22/11/20 18:55:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/20 18:55:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load the dataframes

In [4]:
movies_df = spark.read.load("/home/sahil/Desktop/Fall 22/CS 532/project/ml-latest/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/home/sahil/Desktop/Fall 22/CS 532/project/ml-latest/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/home/sahil/Desktop/Fall 22/CS 532/project/ml-latest/links.csv", format='csv', header = True)
tags_df = spark.read.load("/home/sahil/Desktop/Fall 22/CS 532/project/ml-latest/tags.csv", format='csv', header = True)

In [5]:
type(movies_df)

pyspark.sql.dataframe.DataFrame

### Show the dataframes and make the lifetime of dataframes sames as spark session

In [6]:
movies_df.show(20)

movies_df.createOrReplaceTempView("movies_df")

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [7]:
ratings_df.show(20)
ratings_df.createOrReplaceTempView("ratings_df")

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
|     1|   1590|   2.5|1256677236|
|     1|   1591|   1.5|1256677475|
|     1|   2134|   4.5|1256677464|
|     1|   2478|   4.0|1256677239|
|     1|   2840|   3.0|1256677500|
|     1|   2986|   2.5|1256677496|
|     1|   3020|   4.0|1256677260|
|     1|   3424|   4.5|1256677444|
|     1|   3698|   3.5|1256677243|
|     1|   3826|   2.0|1256677210|
|     1|   3893|   3.5|1256677486|
|     2|    170|   3.5|1192913581|
|     2|    849|   3.5|1192913537|
|     2|   1186|   3.5|1192913611|
|     2|   1235|   3.0|1192913585|
+------+-------+------+----------+
only showing top 20 rows



In [8]:
links_df.show(20)

links_df.createOrReplaceTempView("links_df")

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
|      6|0113277|   949|
|      7|0114319| 11860|
|      8|0112302| 45325|
|      9|0114576|  9091|
|     10|0113189|   710|
|     11|0112346|  9087|
|     12|0112896| 12110|
|     13|0112453| 21032|
|     14|0113987| 10858|
|     15|0112760|  1408|
|     16|0112641|   524|
|     17|0114388|  4584|
|     18|0113101|     5|
|     19|0112281|  9273|
|     20|0113845| 11517|
+-------+-------+------+
only showing top 20 rows



In [9]:
tags_df.show(20)

tags_df.createOrReplaceTempView("tags_df")

+------+-------+--------------+----------+
|userId|movieId|           tag| timestamp|
+------+-------+--------------+----------+
|    14|    110|          epic|1443148538|
|    14|    110|      Medieval|1443148532|
|    14|    260|        sci-fi|1442169410|
|    14|    260|  space action|1442169421|
|    14|    318|  imdb top 250|1442615195|
|    14|    318|       justice|1442615192|
|    14|    480|     Dinosaurs|1443148563|
|    14|    593|psychothriller|1444014286|
|    14|   1682|    philosophy|1442615158|
|    14|   1682|  surveillance|1442615167|
|    14|   7458|          Epic|1443148675|
|    14|  95311|         Pixar|1443148494|
|    14| 117529|     dinosaurs|1443148646|
|    27|    260|classic sci-fi|1440448113|
|    27|    260| Harrison Ford|1440448123|
|    27|    260|      must see|1440448132|
|    27|    260|        sci-fi|1440448094|
|    42|  37733| disappointing|1264106059|
|    42|  37733|     overrated|1264106052|
|    42|  37733|        stupid|1264106067|
+------+---

### Registering the dataframes to spark 

In [10]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

/home/sahil/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


### Analyse the data

In [11]:
minRating_1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
minRating_2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()

print('Minimum number of ratings per user: {}'.format(minRating_1))
print('Minimum number of ratings per movie: {}'.format(minRating_2))

Minimum number of ratings per user: 1
Minimum number of ratings per movie: 1


In [12]:
_rating1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
_total = ratings_df.select('movieId').distinct().count()

print('movies are rated by only one user: {} out of {} '.format(_rating1, _total))

movies are rated by only one user: 10155 out of 53889 


In [13]:
# number of distinct users
num_users = spark.sql("SELECT count (distinct userID) as num_users FROM ratings")
ratings_df.select("userId").distinct().count()

283228

In [14]:
# number of movies
num_movies = spark.sql("SELECT count (distinct movieID) as num_movies FROM movies")
print(movies_df.select('movieID').distinct().count())

58098


In [15]:
rated_by_users = ratings_df.select('movieID').distinct().count()
print('Total Number of movies rated by users:', rated_by_users)

Total Number of movies rated by users: 53889


In [16]:
# null rated movies
spark.sql("SELECT movies.title, movies.genres ,ratings.rating FROM movies left JOIN ratings ON ratings.movieId = movies.movieID WHERE ratings.rating IS null LIMIT 10").show()

+--------------------+--------------------+------+
|               title|              genres|rating|
+--------------------+--------------------+------+
|   Fambul Tok (2011)|         Documentary|  null|
|Shadow Boxers (1999)|         Documentary|  null|
| 9500 Liberty (2009)|         Documentary|  null|
|  Ascent, The (1994)|       Adventure|War|  null|
|Laffghanistan: Co...|Comedy|Documentar...|  null|
|      Annie O (1996)|       Drama|Romance|  null|
|Cry in the Night,...|      Drama|Thriller|  null|
|Across the Sierra...|Action|Romance|We...|  null|
|Bachelor Bait (1934)|      Comedy|Romance|  null|
|Back in the Saddl...|Action|Drama|Western|  null|
+--------------------+--------------------+------+



In [17]:
# movie genres
spark.sql("SELECT DISTINCT(genres) FROM movies LIMIT 10").show()

+--------------------+
|              genres|
+--------------------+
|Comedy|Horror|Thr...|
|Adventure|Sci-Fi|...|
|Action|Adventure|...|
| Action|Drama|Horror|
|Comedy|Drama|Horr...|
|Action|Animation|...|
|Fantasy|Musical|M...|
|Adventure|Mystery...|
|Children|Comedy|D...|
|Action|Adventure|...|
+--------------------+



In [18]:
extract_genres = udf(lambda x: x.split("|"), ArrayType(StringType()))
movies_df_clean = movies_df.select("movieId", "title", extract_genres("genres").alias("genres"))

movies_df_clean.createOrReplaceTempView("movies_df_clean")

display (spark.sql("SELECT * FROM movies_df_clean limit 5"))

DataFrame[movieId: string, title: string, genres: array<string>]

In [19]:
movies_df_clean.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|[Adventure, Anima...|
|      2|      Jumanji (1995)|[Adventure, Child...|
|      3|Grumpier Old Men ...|   [Comedy, Romance]|
|      4|Waiting to Exhale...|[Comedy, Drama, R...|
|      5|Father of the Bri...|            [Comedy]|
+-------+--------------------+--------------------+
only showing top 5 rows



In [20]:
# All movie categories
genres_result = list(set(movies_df_clean.select('genres').rdd.flatMap(tuple).flatMap(tuple).collect()))
genres_result

['IMAX',
 '(no genres listed)',
 'Sci-Fi',
 'Crime',
 'Mystery',
 'Horror',
 'Children',
 'Adventure',
 'Animation',
 'Thriller',
 'Documentary',
 'Fantasy',
 'Romance',
 'War',
 'Film-Noir',
 'Comedy',
 'Western',
 'Drama',
 'Musical',
 'Action']

In [21]:
movie_pdf = movies_df.toPandas()
list_of_movie = list(movie_pdf['title'])

In [22]:
# Data type convert
movie_ratings=ratings_df.drop('timestamp')

movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [23]:
movie_ratings.show(50)
movie_ratings.createOrReplaceTempView("movie_ratings")
display (spark.sql("SELECT * FROM movie_ratings limit 10"))

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    307|   3.5|
|     1|    481|   3.5|
|     1|   1091|   1.5|
|     1|   1257|   4.5|
|     1|   1449|   4.5|
|     1|   1590|   2.5|
|     1|   1591|   1.5|
|     1|   2134|   4.5|
|     1|   2478|   4.0|
|     1|   2840|   3.0|
|     1|   2986|   2.5|
|     1|   3020|   4.0|
|     1|   3424|   4.5|
|     1|   3698|   3.5|
|     1|   3826|   2.0|
|     1|   3893|   3.5|
|     2|    170|   3.5|
|     2|    849|   3.5|
|     2|   1186|   3.5|
|     2|   1235|   3.0|
|     2|   1244|   3.0|
|     2|   1296|   4.5|
|     2|   1663|   3.0|
|     2|   1962|   4.0|
|     2|   2108|   3.5|
|     2|   2243|   4.5|
|     2|   2352|   4.0|
|     2|   2707|   3.5|
|     2|   2746|   4.0|
|     2|   2915|   3.5|
|     2|   3363|   4.0|
|     3|    640|   3.0|
|     3|    828|   4.0|
|     3|    960|   3.0|
|     3|   1221|   4.0|
|     3|   1321|   4.0|
|     3|   1645|   4.0|
|     3|   1825|   3.0|
|     3|   1985|

DataFrame[userId: int, movieId: int, rating: float]

In [38]:
movie_rating_sample = movie_ratings.sample(False, 1/500)
movie_rating_sample.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   2986|   2.5|
|     4|    292|   3.5|
|     4|    836|   2.5|
|    10|   1093|   2.0|
|    14| 160718|   4.5|
|    19|    379|   4.0|
|    55|   1431|   1.5|
|    56|   2005|   1.0|
|    56|   3255|   1.0|
|    56|  91542|   1.5|
|    70|    736|   4.0|
|    72|   1213|   3.5|
|    81|  61248|   3.0|
|    82|   2968|   4.5|
|    88|   4226|   4.0|
|   111|    912|   5.0|
|   114|    539|   4.0|
|   134|    611|   2.5|
|   134|   4016|   4.0|
|   134| 128838|   3.0|
+------+-------+------+
only showing top 20 rows

